In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


# Module

In [2]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

# Dataset

In [3]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [4]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [5]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [6]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [8]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [9]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [10]:
model.fit(x_train_reshaped, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 6s 108us/sample - loss: 0.1941 - accuracy: 0.9417
Epoch 2/10
60000/60000 [==============================] - 3s 57us/sample - loss: 0.0646 - accuracy: 0.9796
Epoch 3/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0454 - accuracy: 0.9857
Epoch 4/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.0361 - accuracy: 0.9885
Epoch 5/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.0289 - accuracy: 0.9913
Epoch 6/10
60000/60000 [==============================] - 3s 55us/sample - loss: 0.0232 - accuracy: 0.9926
Epoch 7/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0189 - accuracy: 0.9941
Epoch 8/10
60000/60000 [==============================] - 3s 54us/sample - loss: 0.0156 - accuracy: 0.9949
Epoch 9/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0129 - accuracy: 0.9957
Epoch 10/10
6

# Model Evaluation
### 1) 원본 모델 -> 원본 이미지

In [11]:
predictions_model = model.predict(x_test_reshaped)
accuracy_model = np.sum(np.argmax(predictions_model, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy_model*100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on benign test example: 98.91%


# Adversarial Examples

## PGD 공격
### 2) 원본 모델 -> 적대적 예제 이미지

In [12]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=False)

In [55]:
from art.attacks.evasion import ProjectedGradientDescent

# eps=0.2,eps_step = 0.5,max_iter=100,batch_size = 100
#Create PGD Instance
attack_pgd = ProjectedGradientDescent(estimator, eps=0.2,
                                     eps_step=0.5,max_iter=100,
                                      batch_size=100)

#Create Adversarial Examples
examples_pgd = attack_pgd.generate(x_test_reshaped,y_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [56]:
# 기존 모델 -> PGD 공격
prediction_pgd = model.predict(examples_pgd)

accuracy_pgd = np.sum(np.argmax(prediction_pgd, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_pgd * 100))

Accuracy on Adversarial Exmaples: 14.64%


# Adversarial Defense (1) 
### user_logits = True


In [13]:
from art.defences.trainer import AdversarialTrainerMadryPGD

In [14]:
defense = AdversarialTrainerMadryPGD(estimator,nb_epochs=391,
                                     batch_size=516,eps=8,eps_step=2,
                                     max_iter=7,num_random_init=1)

In [24]:
defense.fit(x_train_reshaped, y_train)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/391 [00:00<?, ?it/s]

In [31]:
# 방어 모델로 생성됨
new_estimator = defense.get_classifier()

### 3-1) 방어 모델 -> 원본 이미지 예측

In [33]:
# 방어모델 생성 후 원본 이미지 예측 결과
prediction_def = new_estimator.predict(x_test_reshaped)

accuracy_def = np.sum(np.argmax(prediction_def, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_def * 100))

Accuracy on Adversarial Exmaples: 93.24%


### 4-1) 방어 모델 -> PGD 공격 된 적대적 예제 예측

In [35]:
# 방어모델 생성 후 pgd로 공격된 예측 결과
prediction_def = new_estimator.predict(examples_pgd)

accuracy_def = np.sum(np.argmax(prediction_def, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_def * 100))

Accuracy on Adversarial Exmaples: 81.93%


### 4-2) 방어 모델을 공격에 적용

In [36]:
from art.attacks.evasion import ProjectedGradientDescent

# eps=0.2,eps_step = 0.5,max_iter=100,batch_size = 100 = 24.65%
#Create PGD Instance
attack_pgd2 = ProjectedGradientDescent(new_estimator, eps=0.2,
                                     eps_step=0.5,max_iter=100,
                                      batch_size=100)

#Create Adversarial Examples
examples_pgd2 = attack_pgd2.generate(x_test_reshaped,y_test)

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

PGD - Random Initializations:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
# 기존 모델 -> PGD 공격
prediction_pgd2 = model.predict(examples_pgd2)

accuracy_pgd2 = np.sum(np.argmax(prediction_pgd2, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_pgd2 * 100))

Accuracy on Adversarial Exmaples: 45.190000000000005%


# 방어 모델 -> FGSM 공격하기

# FastGradientMethod

1) 원본 모델 -> 원본 이미지 예측  
    - 98.98%  
2) 원본 모델 -> 적대적예제 예측  
3) 방어 모델 -> 원본 이미지 예측   
4) _1 방어모델 -> 적대적 예제 예측  
4) _2 방어 모델을 공격

In [59]:
# ART
import art
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=False)

In [61]:
#FGSM ver2

#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.3)

#Create Adversarial Examples
x_attack_fgsm = attack_fgsm.generate(x_test_reshaped)

In [62]:
prediction_fgsm = model.predict(x_attack_fgsm)

accuracy_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_fgsm * 100))

Accuracy on Adversarial Exmaples: 20.419999999999998%


In [66]:
# FGSM ver4-1
# 방어모델 생성 후 pgd로 공격된 예측 결과
prediction_def = new_estimator.predict(x_attack_fgsm)

accuracy_def = np.sum(np.argmax(prediction_def, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_def * 100))

Accuracy on Adversarial Exmaples: 67.17%


In [64]:
#FGSM ver4-2
#Create FGSM Instance
attack_fgsm2 = FastGradientMethod(new_estimator, eps=0.3)

#Create Adversarial Examples
x_attack_fgsm2 = attack_fgsm2.generate(x_test_reshaped)

In [65]:
prediction_fgsm2 = model.predict(x_attack_fgsm2)

accuracy_fgsm2 = np.sum(np.argmax(prediction_fgsm2, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_fgsm2 * 100))

Accuracy on Adversarial Exmaples: 76.66%


# JSMA

In [67]:
y_test_mix = y_test.copy()

In [71]:
np.random.shuffle(y_test_mix)
y_test_mix

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [84]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [72]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator_jsma = KerasClassifier(model=model, clip_values=(0,1), use_logits=False)

In [73]:
from art.attacks.evasion import SaliencyMapMethod
#JSMS ver2
#Create jsma Instance
attack_jsma = SaliencyMapMethod(estimator_jsma, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma = attack_jsma.generate(x_test_reshaped)

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

1) 원본 모델 -> 원본 이미지 예측  
    - 98.98%  
2) 원본 모델 -> 적대적예제 예측  
3) 방어 모델 -> 원본 이미지 예측   
4) _1 방어모델 -> 적대적 예제 예측  
4) _2 방어 모델을 공격

In [74]:
# JSMA ver 2
# 원본 모델 -> 적대적 예제 예측 (untargeted)

loss,acc = model.evaluate(x_attack_jsma,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on Adversarial Exmaples: 9.870000183582306%


In [82]:
# JSMA ver3
prediction_def = new_estimator.predict(x_test_reshaped)

accuracy_def = np.sum(np.argmax(prediction_def, axis=1) == np.argmax(y_test_mix, axis=1)) / len(y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_def * 100))

Accuracy on Adversarial Exmaples: 10.42%


In [78]:
# JSMA ver4-1
prediction_def = new_estimator.predict(x_attack_jsma)

accuracy_def = np.sum(np.argmax(prediction_def, axis=1) == np.argmax(y_test_mix, axis=1)) / len(y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_def * 100))

Accuracy on Adversarial Exmaples: 10.12%


In [79]:
#JSMA ver4-2
#Create jsma Instance
attack_jsma2 = SaliencyMapMethod(new_estimator, theta=0.3,
                                     gamma=0.1,batch_size=50)

#Create Adversarial Examples
x_attack_jsma2 = attack_jsma2.generate(x_test_reshaped)

JSMA:   0%|          | 0/200 [00:00<?, ?it/s]

In [83]:
loss,acc = model.evaluate(x_attack_jsma2,y_test_mix)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc* 100))

Accuracy on Adversarial Exmaples: 10.14000028371811%


In [81]:
# JSMA
# 기존 모델 -> jsma로 공격
prediction_jsma2 = model.predict(x_attack_jsma2)

accuracy_jsma2 = np.sum(np.argmax(prediction_jsma2, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(accuracy_jsma2 * 100))

Accuracy on Adversarial Exmaples: 91.69%
